# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921718131261                   -0.69    6.0    353ms
  2   -7.926162680975       -2.35       -1.22    1.0    172ms
  3   -7.926838089337       -3.17       -2.37    1.9    221ms
  4   -7.926861198956       -4.64       -3.01    2.2    235ms
  5   -7.926861638096       -6.36       -3.34    1.9    202ms
  6   -7.926861665155       -7.57       -3.70    1.4    192ms
  7   -7.926861679484       -7.84       -4.17    1.2    167ms
  8   -7.926861681801       -8.64       -5.18    1.8    185ms
  9   -7.926861681863      -10.21       -5.21    3.2    247ms
 10   -7.926861681872      -11.01       -6.52    1.0    163ms
 11   -7.926861681873      -12.53       -6.77    3.2    285ms
 12   -7.926861681873      -14.35       -7.18    1.1    173ms
 13   -7.926861681873      -15.05       -7.76    1.5    174ms
 14   -7.926861681873   +  -15.05       -8.90    2.2    209ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921708860780                   -0.69    6.2    335ms
  2   -7.926164929405       -2.35       -1.22    1.0    230ms
  3   -7.926839711314       -3.17       -2.37    1.9    206ms
  4   -7.926861179590       -4.67       -3.06    2.2    237ms
  5   -7.926861660812       -6.32       -3.46    2.0    232ms
  6   -7.926861674696       -7.86       -3.96    1.5    177ms
  7   -7.926861677442       -8.56       -4.00    2.0    222ms
  8   -7.926861679050       -8.79       -4.10    1.0    168ms
  9   -7.926861681763       -8.57       -4.51    1.0    167ms
 10   -7.926861681856      -10.03       -4.71    1.0    169ms
 11   -7.926861681869      -10.88       -5.34    1.0    168ms
 12   -7.926861681872      -11.51       -6.28    2.5    202ms
 13   -7.926861681873      -12.54       -5.71    2.8    246ms
 14   -7.926861681873      -13.06       -6.89    1.0    167ms
 15   -7.

The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921710545474                   -0.69    6.0    401ms
  2   -7.926166460746       -2.35       -1.22    1.0    182ms
  3   -7.926839965673       -3.17       -2.37    1.9    236ms
  4   -7.926864598483       -4.61       -3.01    2.1    255ms
  5   -7.926865046123       -6.35       -3.33    1.8    218ms
  6   -7.926865074823       -7.54       -3.68    1.5    210ms
  7   -7.926865090391       -7.81       -4.14    1.4    175ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
